In [1]:
!pip install -r requeminents.txt

In [2]:
import telebot
from telebot import types
import logging
import gc
from game import *
from model import *

In [3]:
def game_st(agent):
  results = [agent.game_now() for i in range(10000)]
  game_stats = {k: results.count(k)/100 for k in ["X", "0", '-']}
  print("Статистика обучения: {}".format(game_stats))

def to_numb(x):
  x = x.lower()
  if len(x) != 2 or x[1].isalpha() or int(x[1]) > 3 :
    return 10
  if x[0] == "a":
    return int(x[1]) - 1
  if x[0] == "b":
    return 2 + int(x[1])
  if x[0] == "c":
    return 5 + int(x[1])
  return 10

In [4]:
agent = Agent007(Crosses_zeros)
print('Before:')
game_st(agent)

agent.learn(10000)
print('After:')
game_st(agent)

Before:
Статистика обучения: {'X': 58.07, '0': 28.91, '-': 13.02}
After:
Статистика обучения: {'X': 1.86, '0': 0.61, '-': 97.53}


In [5]:
player = dict()
game = dict()


bot = telebot.TeleBot('1818851136:AAHzS5ZKjx_GNfe5V_LhpdnB95lWlXW9E08')
@bot.message_handler(content_types=['text'])
def get_text_messages(message):
  if message.text.lower() == "привет":
    bot.send_message(message.from_user.id, "Привет, хочешь поиграть?")
    bot.register_next_step_handler(message, continue_bot);
  elif message.text == "/help":
    bot.send_message(message.from_user.id, "Напиши привет")
  else:
    bot.send_message(message.from_user.id, "Я тебя не понимаю. Напиши /help.")

def continue_bot(message):
  if (message.text).lower() == 'да':
    figure(message)
  else:
    bot.send_message(message.from_user.id, "Тогда пока!")

def figure(message):
      keyboard = types.InlineKeyboardMarkup(); #наша клавиатура
      key_yes = types.InlineKeyboardButton(text='X', callback_data="1"); #кнопка «Да»
      keyboard.add(key_yes); #добавляем кнопку в клавиатуру
      key_yes2 = types.InlineKeyboardButton(text='0', callback_data="0");
      keyboard.add(key_yes2);
      question = 'Что ты выберешь?';
      bot.send_message(message.from_user.id, text=question, reply_markup=keyboard)

@bot.callback_query_handler(func=lambda call: True)
def callback_worker(call):
    bot.send_message(call.message.chat.id, 'Хороший выбор! Если хочешь, чтобы я тебя понял, свой ход делай с помощью команд: a1, a2, a3, b1, b2, b3, c1, c2, c3, где a, b, c - строки, а 1, 2, 3 - столбцы ')
    global player, game
    game[call.message.chat.id] = Crosses_zeros()
    player[call.message.chat.id] = int(call.data)
    if  player[call.message.chat.id] == 0:
        game[call.message.chat.id].do_action(agent.select_action_for_interactive_game(game[call.message.chat.id], 1))
        bot.send_message(call.message.chat.id, game[call.message.chat.id].table_to_string(), parse_mode="HTML")
    bot.register_next_step_handler(call.message, agent_play);

def agent_play(message):
    global player, game
    now = to_numb(message.text)
    table = game[message.chat.id].table
    if now < 0 or now > 8 or table[now] != 1337:
      bot.reply_to(message, "Некорректный ход")
    else:
      table[now] = player[message.chat.id]
      game[message.chat.id].do_action(table)
      if game[message.chat.id].playable():
        game[message.chat.id].do_action(agent.select_action_for_interactive_game(game[message.chat.id], 1 - player[message.chat.id]))
      bot.send_message(message.chat.id, game[message.chat.id].table_to_string(), parse_mode="HTML")
    if game[message.chat.id].playable():
      bot.register_next_step_handler(message, agent_play)
    else:
      if game[message.chat.id].check_winner() == 'X':
        bot.send_message(message.chat.id, 'Выиграл X!!!', parse_mode="HTML")
      elif game[message.chat.id].check_winner() == '0':
        bot.send_message(message.chat.id, 'Выиграл 0!!!', parse_mode="HTML")
      else:
        bot.send_message(message.chat.id, 'Ничья!!!', parse_mode="HTML")


bot.polling(none_stop=True, interval=0)
